In [74]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


env = gym.make("CartPole-v1")

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [3]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [4]:
from torch.utils.tensorboard import SummaryWriter

# # Initialize the network and the TensorBoard writer
# dqn = DQN(n_observations, n_actions)
# writer = SummaryWriter()

# # Assume input is a tensor of observations
# input = torch.randn(n_observations)

# # Forward pass through the network
# output = dqn(input)

# # Log the output tensor to TensorBoard
# writer.add_scalar('Output', output, global_step)

# # Don't forget to close the writer when you're done
# writer.close()

In [5]:
inputs = torch.randn(100, 3)  #100 num of cars, 3 for each car coordinates start, stop, and tmie to arrive



In [6]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

# Get number of actions from gym action space
n_actions = env.action_space.n      #time for start drive 
# Get the number of state observations
state, info = env.reset()
n_observations = len(state)

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)


episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [7]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [8]:
from torch.utils.tensorboard import SummaryWriter

# Create a SummaryWriter
writer = SummaryWriter()

if torch.cuda.is_available():
    num_episodes = 600
else:
    num_episodes = 50

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            # Log the episode duration to TensorBoard
            writer.add_scalar('Episode duration', t + 1, i_episode)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

# Close the SummaryWriter
writer.close()

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [29]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [30]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [31]:
actions= torch.randn(100, 12)  #100 num of cars, 1 for each car action to start drivi
n_actions = len(actions[0]) * len(actions[1])

state = torch.randn(100, 4)  #100 num of cars, 3 for each car coordinates start, stop, and tmie to arrive
n_observations = len(state)*len(state[0])
print(n_observations, n_actions)

400 144


In [32]:
add_time=np.linspace(-30, 30, 13)
add_time

array([-30., -25., -20., -15., -10.,  -5.,   0.,   5.,  10.,  15.,  20.,
        25.,  30.])

In [34]:
import pandas as pd

In [35]:
# Create a DataFrame 100x 4 list corStart corsStop timestart timeStop

df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=['corStart', 'corsStop', 'timestart', 'timeStop'])

In [36]:
df.head()

,corStart,corsStop,timestart,timeStop
0,48,31,82,37
1,12,59,70,22
2,58,26,17,43
3,9,99,37,75
4,48,24,87,91


In [49]:
# dataset in class pytroch
from torch.utils.data import Dataset

class trips_data_set(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

In [38]:
#df to trio data set
trips_dataset = trips_data_set(df)


In [39]:
#polot df as line x is A and y is sum C and D
df['sum'] = df['timestart'] + df['timeStop']


In [40]:
df.head()


,corStart,corsStop,timestart,timeStop,sum
0,48,31,82,37,119
1,12,59,70,22,92
2,58,26,17,43,60
3,9,99,37,75,112
4,48,24,87,91,178


In [41]:
vec_of_sum = np.zeros(10)

vec_of_sum.shape
vec_of_sum[1] += float(df["sum"][2])


In [42]:

for i in range(0,len(df)):
    if df["timeStop"][i] > 0 and df["timeStop"][i] < 10:
        vec_of_sum[0] += df["sum"][i]   
    elif df["timeStop"][i] > 10 and df["timeStop"][i] < 20:
        vec_of_sum[1] += df["sum"][i] 
    elif df["timeStop"][i] > 20 and df["timeStop"][i] < 30:
        vec_of_sum[2] += df["sum"][i] 
    elif df['timeStop'][i] > 30 and df['timeStop'][i] < 40:
        vec_of_sum[3] += df["sum"][i] 
    elif df['timeStop'][i] > 40 and df['timeStop'][i] < 50:
        vec_of_sum[4] += df["sum"][i] 
    elif df['timeStop'][i] > 50 and df['timeStop'][i] < 60:
        vec_of_sum[5] += df["sum"][i] 
    elif df["timeStop"][i] > 60 and df['timeStop'][i] < 70:
        vec_of_sum[6] += df["sum"][i] 
    elif df['timeStop'][i] > 70 and df['timeStop'][i] < 80:
        vec_of_sum[7] += df["sum"][i] 
    elif df["timeStop"][i] > 80 and df['timeStop'][i] < 90:
        vec_of_sum[8] += df["sum"][i] 
    elif df['timeStop'][i] > 90 and df['timeStop'][i] < 100:
        vec_of_sum[9] += df["sum"][i] 
 


In [43]:
import pandas as pd
import numpy as np

def sum_by_time_intervals(df, time_col_name, sum_col_name, interval):
    # Oblicz maksymalną wartość w kolumnie czasowej, aby określić, ile przedziałów potrzebujemy
    max_time = df[time_col_name].max()
    # Obliczamy liczbę przedziałów na podstawie maksymalnego czasu i długości przedziału
    num_intervals = int(np.ceil(max_time / interval))
    
    # Tworzymy wektor zer o długości liczby przedziałów do przechowywania sum
    vec_of_sum = np.zeros(num_intervals)
    
    # Iterujemy przez każdy wiersz DataFrame
    for i in range(len(df)):
                # Oblicz indeks przedziału dla aktualnego wiersza
        interval_index = int(df[time_col_name][i] // interval)
        # Upewnij się, że nie przekraczamy zakresu wektora
        if interval_index < num_intervals:
            # Dodaj wartość z kolumny sum do odpowiedniego przedziału
            vec_of_sum[interval_index] += df[sum_col_name][i]
            
    return vec_of_sum


In [45]:
#load csv to df
df = pd.read_csv('dataset_to_train.csv')

In [46]:
df.head()

,interaction,vehicle_id,x_start,y_start,x_end,y_end,startTime,arriveTime
0,0,0,22.014639,50.021600,22.015684,50.021603,0,14
1,0,2,22.015553,50.028272,22.018961,50.028306,0,153
2,1,2,22.015553,50.028272,22.018961,50.028306,0,153
3,1,3,22.015553,50.028272,22.018967,50.028306,0,82
4,1,0,22.014639,50.021600,22.015684,50.021603,0,14


In [ ]:
x= sum_by_time_intervals(df, 'timeStop', 'sum', 10)

In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    54 = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [ ]:
def optimize_model_for_uniformity():
    # Załóżmy, że mamy już obliczone sumy dla przedziałów czasowych za pomocą sum_by_time_intervals
    # Tutaj możemy obliczyć wektor sum 'vec_of_sum' albo przekazać go jako argument funkcji
    
    if len(memory) < BATCH_SIZE:
        return
    
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))

    # Przykład obliczenia 'vec_of_sum' z danych w 'batch'
    # To tylko przykład, rzeczywiste obliczenie zależy od Twojej implementacji sum_by_time_intervals
    # i jak przechowujesz dane w 'memory'
    vec_of_sum = np.zeros(10) # Zakładamy 10 przedziałów czasowych jako przykład
    
    for transition in transitions:
        
        pass

    # Obliczanie miary nierównomierności, np. wariancji
    uniformity_measure = np.var(vec_of_sum)
    
    # Konwersja na tensor PyTorch i obliczenie funkcji straty jako negatywnej miary nierównomierności
    # Celem jest maksymalizacja równomierności, więc minimalizujemy negatywną wartość
    loss = -torch.tensor(uniformity_measure, device=device)

    # Optymalizacja modelu
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
!nvidia-smi

In [ ]:
def model(df):

    return df[]

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# Create a SummaryWriter
writer = SummaryWriter()

if torch.cuda.is_available():
    num_episodes = 600
else:
    num_episodes = 50

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            # Log the episode duration to TensorBoard
            writer.add_scalar('Episode duration', t + 1, i_episode)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

# Close the SummaryWriter
writer.close()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x4 and 400x128)

In [51]:
class DQN(nn.Module):
    def __init__(self, inputs, outputs):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(inputs, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, outputs)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [52]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_action_values = policy_net(state_batch).gather(1, action_batch)
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    loss = nn.SmoothL1Loss()(state_action_values, expected_state_action_values.unsqueeze(1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [53]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)


In [64]:
# Create a DataFrame 100x 4 list corStart corsStop timestart timeStop

df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=['corStart', 'corsStop', 'timestart', 'timeStop'])
df

,corStart,corsStop,timestart,timeStop
0,33,54,38,61
1,32,14,87,19
2,86,46,62,36
3,83,76,55,66
4,43,75,39,19
...,...,...,...,...
95,19,56,53,80
96,88,0,11,98
97,41,43,55,79
98,14,4,32,48


In [66]:
# add new comumn sum of time start and time stop
df['sum'] = df['timestart'] + df['timeStop']

In [62]:
import pandas as pd
import numpy as np

def sum_by_time_intervals(df, time_col_name, sum_col_name, interval):
    # Oblicz maksymalną wartość w kolumnie czasowej, aby określić, ile przedziałów potrzebujemy
    max_time = df[time_col_name].max()
    # Obliczamy liczbę przedziałów na podstawie maksymalnego czasu i długości przedziału
    num_intervals = int(np.ceil(max_time / interval))
    
    # Tworzymy wektor zer o długości liczby przedziałów do przechowywania sum
    vec_of_sum = np.zeros(num_intervals)
    
    # Iterujemy przez każdy wiersz DataFrame
    for i in range(len(df)):
                # Oblicz indeks przedziału dla aktualnego wiersza
        interval_index = int(df[time_col_name][i] // interval)
        # Upewnij się, że nie przekraczamy zakresu wektora
        if interval_index < num_intervals:
            # Dodaj wartość z kolumny sum do odpowiedniego przedziału
            vec_of_sum[interval_index] += df[sum_col_name][i]
            
    return vec_of_sum


In [70]:
#convert df to tensor witchout sum
inputs = torch.tensor(df.drop(columns=['sum']).values, dtype=torch.float32)
outputs = torch.tensor(df['timestart'].values, dtype=torch.float32)

In [67]:
sum = sum_by_time_intervals(df, 'timeStop', 'sum', 10)


In [ ]:
torch.ran

In [54]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

policy_net = DQN(len(inputs[0])*len(inputs[1]), len(outputs)).to(device)
target_net = DQN(len(inputs[0])*len(inputs[1]), len(outputs)).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters())
memory = ReplayMemory(10000)

steps_done = 0

def select_action(state):
    global steps_done
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if random.random() > eps_threshold:
        with torch.no_grad():
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(env.action_space.n)]], device=device, dtype=torch.long)


In [55]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_action_values = policy_net(state_batch).gather(1, action_batch)
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    loss = nn.SmoothL1Loss()(state_action_values, expected_state_action_values.unsqueeze(1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
